# Repair Our Air



## О компании

Repair Our Air (ROA) разрабатывает рекомендации по улучшению качества воздуха в Америке, используя для принятия решений индекс качества воздуха (AQI), разработанный Агентством по охране окружающей среды. Значение AQI, близкое к 0, свидетельствует о "незначительной или нулевой" угрозе здоровью населения, в то время как более высокие значения связаны с повышенным риском для здоровья.

## Метаданные

| Имя колонки      | Тип      | Описание                                                                                                                                                                                                                                                             |
|------------------|----------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| [None]           | int      | Индекс                                                                                                                                                                                                                                                               |
| date_local       | datetime | Дата снятия показаний AQI                                                                                                                                                                                                                                            |
| state_name       | str      | Название штата США                                                                                                                                                                                                                                                   |
| county_name      | str      | Название округа США                                                                                                                                                                                                                                                  |
| city_name        | str      | Город США, в котором проводился мониторинг                                                                                                                                                                                                                           |
| local_site name  | str      | Идентификатор объекта в реестре агентства-владельца (например, не US EPA)                                                                                                                                                                                            |
| parameter_name   | str      | Описание параметра в системе качества воздуха, измеряемого монитором, включая загрязняющие и незагрязняющие вещества (например, скорость ветра)                                                                                                                      |
| units_of_measure | str      | Единица измерения для всех данных. Каждый параметр имеет стандартную единицу измерения.                                                                                                                                                                              |
| aritmetic_mean   | int      | Среднее значение загрязняющих веществ в квартальном наборе данных, рассчитываемое как сумма значений, поделенная на количество соответствующих наблюдений. Для критериальных загрязнителей учитываются только значения с соответствующей маркировкой и типом данных. |
| aqi              | int      | "Индекс качества воздуха (AQI) подобен термометру от 0 до 500. Высокое значение AQI свидетельствует о высоком уровне загрязнения воздуха, что может быть опасным для здоровья."                                                                                      |

**260 строк** - каждая строка представляет собой отдельное показание aqi

**10 колонок**

## Цель проекта

Необходимо использовать данные AQI для проверки гипотез стратегии по улучшению качества воздуха.

1. Отличается ли статистически значимо средний показатель AQI в округе Лос-Анджелес от остальных районов Калифорнии?
2. Имеет ли Нью-Йорк более низкий показатель AQI, чем Огайо?
3. Новая политика затронет те штаты, где средний индекс AQI составляет 10 или более. Можно ли вы исключить, что новая политика затронет Мичиган?

## Этап 1: Импорт
- Импорт библиотек
- Загрузка набора данных

#### Импорт библиотек

In [ ]:
# Импорт необходимых пакетов

import pandas as pd
import numpy as np
from scipy import stats

Для данного анализа предоставляется набор данных с национальными измерениями индекса качества воздуха (AQI) по штатам.

#### Загрузка набора данных

In [ ]:
# Загрузка данных из файла с именем 'c4_epa_air_quality.csv'

aqi = pd.read_csv('c4_epa_air_quality.csv')

## Этап 2: Исследование данных
- Понимание переменных

### Вывод основной информации о данных

In [ ]:
# Вывод основной статистической информации о числовых данных:

print(aqi.head())

print(aqi.describe(include='all'))

print(aqi['state_name'].value_counts())

   Unnamed: 0  date_local    state_name   county_name      city_name  \
0           0  2018-01-01       Arizona      Maricopa        Buckeye   
1           1  2018-01-01          Ohio       Belmont      Shadyside   
2           2  2018-01-01       Wyoming         Teton  Not in a city   
3           3  2018-01-01  Pennsylvania  Philadelphia   Philadelphia   
4           4  2018-01-01          Iowa          Polk     Des Moines   

                                     local_site_name   parameter_name  \
0                                            BUCKEYE  Carbon monoxide   
1                                          Shadyside  Carbon monoxide   
2  Yellowstone National Park - Old Faithful Snow ...  Carbon monoxide   
3                             North East Waste (NEW)  Carbon monoxide   
4                                          CARPENTER  Carbon monoxide   

    units_of_measure  arithmetic_mean  aqi  
0  Parts per million         0.473684    7  
1  Parts per million         0.263158 

- Для первой гипотезы у вас есть данные на уровне округов.
- В этом наборе данных Огайо и Нью-Йорк имеют большее число наблюдений, с которыми можно работать.

## Этап 3. Статистические тесты

- Сформулировать нулевую гипотезу и альтернативную гипотезу.
- Установить уровень значимости.
- Определить подходящую процедуру тестирования.
- Вычислить p-значение.
- Сделать вывод.

### Гипотеза 1: Отличается ли статистически значимо средний показатель AQI в округе Лос-Анджелес от остальных районов Калифорнии.

Прежде чем приступить к анализу, будет полезно разбить данные на подгруппы для сравнения.

In [ ]:
# Создание набора данных для каждого сравниваемого образца в тесте

ca_la = aqi[aqi['county_name']=='Los Angeles']
ca_other = aqi[(aqi['state_name']=='California') & (aqi['county_name']!='Los Angeles')]

#### Формулирование гипотезы

Формулирование нулевой и альтернативной гипотезы:

* $H_0$: Разница в среднем показателе AQI между округом Лос-Анджелес и остальной частью Калифорнии отсутствует.
* $H_A$: Существует разница в среднем показателе AQI между округом Лос-Анджелес и остальной частью Калифорнии.


#### Установка уровня значимости

In [ ]:
# Для данного анализа уровень значимости составляет 5%

significance_level = 0.05
significance_level

0.05

#### Определение подходящей процедуры тестирования

В данном случае сравниваются выборочные средние между двумя независимыми выборками. Поэтому будет использован **двухвыборочный T-тест.**

#### Вычиcление p-value

In [ ]:
# Вычисление p-value

stats.ttest_ind(a=ca_la['aqi'], b=ca_other['aqi'], equal_var=False)

TtestResult(statistic=2.1107010796372014, pvalue=0.049839056842410995, df=17.08246830361151)

Поскольку p-значение (0,049) меньше 0,05 (так как уровень значимости равен 5%), отвергаем нулевую гипотезу в пользу альтернативной гипотезы.

Разница в среднем показателе AQI между округом Лос-Анджелес и остальной частью Калифорнии присутствует.

### Гипотеза 2: Имеет ли Нью-Йорк более низкий AQI, чем Огайо?

Прежде чем приступить к анализу, будет полезно разбить данные на подгруппы для сравнения.

In [ ]:
# Создание набора данных для каждого сравниваемого образца в тесте

ny = aqi[aqi['state_name']=='New York']
ohio = aqi[aqi['state_name']=='Ohio']

#### Формулирование гипотезы

**Формулирование нулевой и альтернативной гипотезы**

* $H_0$: Среднее значение AQI в Нью-Йорке больше или равно среднему значению AQI в Огайо.
* $H_A$: Средний индекс AQI в Нью-Йорке **меньше**, чем в Огайо.


#### Уровень значимости сохраняется (на уровне 5%)

#### Определение подходящей процедуры тестирования

Здесь проводится сравнение выборочных средних между двумя независимыми выборками в одном направлении. Поэтому используется **двухвыборочный T-тест**.

#### Вычисление p-value

In [ ]:
# Вычисление p-value

tstat, pvalue = stats.ttest_ind(a=ny['aqi'], b=ohio['aqi'], alternative='less', equal_var=False)
print(tstat)
print(pvalue)

-2.025951038880333
0.03044650269193468


При p-value (0,030) менее 0,05 (так как уровень значимости равен 5%) и T-статистике < 0 (-2,036) отклоним нулевую гипотезу в пользу альтернативной гипотезы.

Таким образом, при 5%-ном уровне значимости можно сделать вывод, что в Нью-Йорке средний показатель AQI ниже, чем в Огайо.

###  Гипотеза 3: Можно ли исключить, что новая политика затронет Мичиган?

Прежде чем приступить к анализу, будет полезно разбить данные на подгруппу для сравнения.

In [ ]:
# Создание набора данных для каждого сравниваемого образца в тесте

michigan = aqi[aqi['state_name']=='Michigan']

#### Формулирование гипотезы

**Формулирование нулевой и альтернативной гипотезы**

* $H_0$: средний показатель AQI Мичигана меньше или равен 10.
* $H_A$: средний показатель AQI Мичигана превышает 10.


#### Уровень значимости сохраняется (на уровне 5%)

#### Определение подходящей процедуры испытания

Здесь проводится сравнение выборочных средних между двумя независимыми выборками в одном направлении. Поэтому используется **двухвыборочный T-тест**.

#### Вычисление P-value

In [ ]:
# Вычисление p-value

tstat, pvalue = stats.ttest_1samp(michigan['aqi'], 10, alternative='greater')
print(tstat)
print(pvalue)

-1.7395913343286131
0.9399405193140109


Если p-value (0,940) превышает 0,05 (поскольку наш уровень значимости составляет 5%) и t-статистика <0 (-1,74), нам не удастся отвергнуть нулевую гипотезу.
Следовательно, на уровне значимости 5% нельзя заключить, что средний показатель AQI Мичигана превышает 10. Это означает, что на Мичиган не повлияет новая политика.

## Этап 4. Результаты и оценка

Теперь, когда мы завершили статистические тесты, можем рассмотреть наши гипотезы и полученные результаты.

- Результаты показали, что AQI в округе Лос-Анджелес на самом деле отличалась от остальной части Калифорнии.
- Используя уровень значимости 5%, на основе результатов можно сделать вывод, что в Нью-Йорке AQI ниже, чем в Огайо.
- На основании проведенных тестов нельзя отклонить нулевую гипотезу. Это означает, что нельзя сделать вывод о том, что среднее значение AQI превышает 10. Следовательно, маловероятно, что внедрение новой политики окажет влияние на показатели Мичигана.

# Заключение


Для каждого теста мы представили нулевую и альтернативную гипотезы, затем описали свой вывод и полученное p-value, на основании которого был сделан этот вывод. Поскольку в настройках t-тестов есть несколько ключевых конфигураций, определяющих интерпретацию результатов, мы указали тип выбранного теста, его однохвостовость или двуххвостовость, а также способ выполнения t-теста из `stats`.

Даже при небольших размерах выборки вариаций данных достаточно, чтобы мы могли сделать статистически значимые выводы. На уровне значимости 5% мы определили, что средний показатель AQI в Лос-Анджелесе статистически отличается от показателя остальной части Калифорнии и что средний показатель AQI в Нью-Йорке действительно ниже, чем в Огайо. Однако на уровне значимости 5% мы не смогли прийти к выводу, что средний показатель AQI в Мичигане превышает 10.